In [1]:
def kfold_model_inproc(model_fold_storage, problem_type, model_type, X, y, Xtest):
    import os
    print("kfold", str(os.getpid()))
    import tensorflow as tf
    from keras import backend as K
    import gc
    from tqdm import tqdm
    import numpy as np
    split = 5
    global pred_dim
    global pred_dim_list
    pred_dim_list = [i if i>0 else X.shape[0] for i in problem_type.pred_dim()]
    pred_dim = tuple(i for i in pred_dim_list)
    global oof_pred
    oof_pred = np.zeros(pred_dim)
    global test_pred
    test_pred_dim_list = [i if i>0 else Xtest.shape[0] for i in problem_type.pred_dim()]
    test_pred_dim = tuple(i for i in test_pred_dim_list)
    test_pred = np.zeros((split,) + test_pred_dim)
    from sklearn.model_selection import StratifiedKFold
    kf = StratifiedKFold(n_splits=split, shuffle=True)
    i_fold=0
    all_train_idx = []
    all_valid_idx = []
    all_model_files = []
    global all_metrics
    all_metrics = {}
    global train_idx
    global valid_idx
    for train_idx, valid_idx in tqdm(kf.split(X, y.argmax(1))):
        all_train_idx.append(train_idx)
        all_valid_idx.append(valid_idx)
    nfolds = len(all_valid_idx)
    for i_fold in range(nfolds):
        train_idx = all_train_idx[i_fold]
        valid_idx = all_valid_idx[i_fold]
        Xt = X.iloc[train_idx]
        yt = y[train_idx]
        Xv = X.iloc[valid_idx]
        yv = y[valid_idx]
        
        model, model_files = model_fold_storage.try_load_model(model_type, i_fold)
        if model is None:
            model = model_type.create()
            model_type.fit(model, Xt, yt, Xv, yv)
            model_files = model_fold_storage.save_model(model_type, i_fold, model)
        
        global oof_df_fold
        oof_df_fold = model_type.predict(model, Xv)
        global metrics
        metrics = model_type.evaluate(model, Xv, yv)
        for key in metrics:
            if key in all_metrics:
                all_metrics[key]=np.hstack((all_metrics[key], metrics[key]))
            else: 
                all_metrics[key]=metrics[key]
        global test_df_fold
        test_df_fold = model_type.predict(model, Xtest)
        model_fold_storage.save_fold(model_type, X, y, valid_idx, train_idx, i_fold, oof_df_fold, test_df_fold, metrics)
        
        oof_pred[valid_idx, :] = np.reshape(oof_df_fold, oof_df_fold.shape)
        test_pred[i_fold]=test_df_fold
        all_model_files.append(model_files)
        i_fold+=1
        K.clear_session()
        del model
        gc.collect()
        K.clear_session()
        gc.collect()
        K.clear_session()
    model_fold_storage.save_kfold(model_type, X, y, all_valid_idx, all_train_idx, all_model_files, oof_pred, test_pred, all_metrics)
    return oof_pred, test_pred, all_metrics

In [2]:
!mkdir kfold_models
!mkdir fold_models
!mkdir models
!mkdir tar_models
class ModelFoldStorage:
    def save_fold(self, model_type, X, y, valid_idx, train_idx, i_fold, oof_df_fold, test_df_fold, metrics):
        import pickle
        foldData = {}
        foldData['X'], foldData['y'],foldData['valid_idx'],foldData['train_idx']=X,y,valid_idx,train_idx
        foldData['oof_df_fold'],foldData['i_fold'], foldData['metrics'], foldData['test_df_fold']=oof_df_fold,i_fold, metrics, test_df_fold
        with open(r"fold_models/"+model_type.name()+"_"+str(i_fold)+"_fold.bin", "wb") as output_file:
            pickle.dump(foldData, output_file)
    def try_load_kfold(self, model_type):
        import pickle
        with open(r"kfold_models/"+model_type.name()+"_kfold.bin", "rb") as input_file:
            modelTypeData = pickle.read(input_file)
            return modelTypeData['oof_pred'], modelTypeData['test_pred'], modelTypeData['all_metrics']
        return None
    def save_kfold(self, model_type, X, y, all_valid_idx, all_train_idx, all_model_files, oof_pred, test_pred, all_metrics):
        import pickle
        modelTypeData = {}
        modelTypeData['X'], modelTypeData['y'],modelTypeData['all_valid_idx'],modelTypeData['all_train_idx'], modelTypeData['all_model_files']=X,y,all_valid_idx,all_train_idx, all_model_files
        modelTypeData['model_type'], modelTypeData['oof_pred'],modelTypeData['all_metrics'], modelTypeData['test_pred']=model_type, oof_pred, all_metrics, test_pred
        with open(r"kfold_models/"+model_type.name()+"_kfold.bin", "wb") as output_file:
            pickle.dump(modelTypeData, output_file)
    def save_model(self, model_type, i_fold, model):
        file = r"models/"+model_type.name()+"_"+str(i_fold)+"_model"
        files = model_type.save(file, model)
        files = self.tar(file, files)
        print("saved model %s to %s (%s)" % (model_type.name(), file, ", ".join(files)))
        return files
    def try_load_model(self, model_type, i_fold):
        file = r"models/"+model_type.name()+"_"+str(i_fold)+"_model"
        file = self.try_untar(file)
        model, model_files = model_type.load(file)
        if model is None:
            print("could not load model %s from %s" % (model_type.name(), file))
        else:
            print("loaded model %s from %s" % (model_type.name(), file))
        return model, model_files
    def tar(self, file, files):
        import tarfile
        tar = "tar_"+file + ".tar.gz.xyz"
        with tarfile.open(tar, "w:gz") as tar:
            for f in files:
                tar.add(f, arcname=os.path.basename(f))
        files.append(tar)
        return files
    def try_untar(self, file):
        import pathlib
        import tensorflow as tf
        import os
        import tarfile
        file_gz = "tar_"+file + ".tar.gz.xyz"
        filepath = pathlib.PurePath(file)
        file_name = filepath.name
        if not os.path.exists(file) and os.path.exists(file_gz):
            import atexit, shutil, tempfile
            models_dir = tempfile.mkdtemp()
            atexit.register(shutil.rmtree, models_dir)
            if not models_dir.endswith('/'):
                models_dir = models_dir + '/'
            print("loading from", file_gz)
            target_model_name = models_dir + file_name
            with tarfile.open(file_gz) as my_tar:
                my_tar.extractall(models_dir) # specify which folder to extract to
                my_tar.close()
            file = target_model_name
            return file
        return file
    
model_fold_storage = ModelFoldStorage()

A subdirectory or file models already exists.


In [3]:
class MyProblem:
    def pred_dim(self):
        return [0]

In [4]:
class MyModel:
    def name(self):
        pass
    def create(self):
        pass
    def fit(self, model, Xt, yt, Xv, yv):
        pass
    def predict(self, model, Xt):
        pass
    def evaluate(self, model, x, y):
        pass
    def save(self, file, model):
        pass
    def load(self, file):
        pass

In [11]:
def DatasetMapFunction(input_ids, attn_masks, labels):
        return {
            'input_ids': input_ids,
            'attention_mask': attn_masks
            }, labels
class MyBertModel(MyModel):
    def __init__(self, debug=False):
        self.debug = debug
    def create(self):
        import tensorflow as tf
        import tensorflow_hub as hub
        import tensorflow_text as text
        from transformers import TFBertModel
        model = TFBertModel.from_pretrained("bert-base-uncased")
        input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
        attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

        bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
        intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
        output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

        discourse_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
        discourse_model.summary()
        from tensorflow.keras.optimizers import Adam
        discourse_model.compile(optimizer=Adam(learning_rate=1e-5, decay=1e-6), 
                        loss='categorical_crossentropy', 
                        metrics=['accuracy'])
        return (discourse_model, self, model)
    def name(self):
        return "bert"
    def fit(self, model, Xt, yt, Xv, yv):
        X_input_ids, X_attn_masks, yt = self._transform(Xt, yt)
        import tensorflow as tf
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, yt))
        dataset = dataset.map(DatasetMapFunction)     # converting to required format for tensorflow dataset
        dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor
        X_val_input_ids, X_val_attn_masks, yv = self._transform(Xv, yv)
        val_dataset = tf.data.Dataset.from_tensor_slices((X_val_input_ids, X_val_attn_masks, yv))
        val_dataset = val_dataset.map(DatasetMapFunction)     # converting to required format for tensorflow dataset
        val_dataset = val_dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor
        epochs = 5
        if self.debug:
            epochs=1
        model[1].history = model[0].fit(dataset,
            steps_per_epoch=200,
            validation_data=val_dataset,
            epochs=epochs)
    def predict(self, model, X):
        X_test_input_ids, X_test_attn_masks, _y = self._transform(X, None)
        labels = model[0].predict([X_test_input_ids, X_test_attn_masks])
        return labels
    def evaluate(self, model, x, y):
        import tensorflow as tf
        X_input_ids, X_attn_masks, y = self._transform(x,y)
        dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, y))
        dataset = dataset.map(DatasetMapFunction)
        dataset = dataset.shuffle(10000).batch(16, drop_remainder=True)
        return model[0].evaluate(dataset, return_dict=True)
    def save(self, file, model):
        import shutil
        import os
        import tensorflow as tf
        import pickle
        ttt1=file
        tf.keras.models.save_model(model[0], ttt1)
        ttt2=file+"_bert"
        tf.keras.models.save_model(model[2], ttt2)
        ttt3=file+"_python"
        with open(ttt3, "wb") as output_file:
            pickle.dump(model[1].__dict__, output_file)
        return [ttt1, ttt2, ttt3, ttt4]
    def load(self, file):
        import os
        import tensorflow as tf
        import pickle
        ttt1=file
        model_0 = tf.keras.models.load_model(ttt1)
        ttt2=file+"_bert"
        model_2 = tf.keras.models.load_model(ttt2)
        ttt3=file+"_python"
        if not os.path.exists(ttt1) or not os.path.exists(ttt2) or not os.path.exists(ttt3):
            return None, None
        #with open(ttt3, "rb") as input_file:
        #    model1_dict = pickle.load(input_file)
        #self.__dict__.update(model1_dict)
        model_files = [ttt1, ttt2, ttt3]
        return (model_0, self, model_2), model_files
    def _encode_data(self, df, ids, masks, tokenizer):
        from tqdm.auto import tqdm
        for i, text in tqdm(enumerate(df['text'])):
            tokenized_text = tokenizer.encode_plus(
                text,
                max_length=256, 
                truncation=True, 
                padding='max_length', 
                add_special_tokens=True,
                return_tensors='tf'
            )
            ids[i, :] = tokenized_text.input_ids
            masks[i, :] = tokenized_text.attention_mask
        return ids, masks
    
    def _transform(self, X,y):
        from transformers import BertTokenizerFast
        tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
        X["text"]=X['discourse_type'] + tokenizer.sep_token+ X['text']
        if self.debug:
            X = X.head(1000)
            if y is not None:
                y = y[0:1000,:]
        
        import numpy as np
        X_input_ids = np.zeros((len(X), 256))
        X_attn_masks = np.zeros((len(X), 256))
        from tqdm.auto import tqdm
        X_input_ids, X_attn_masks = self._encode_data(X, X_input_ids, X_attn_masks, tokenizer)
        return X_input_ids, X_attn_masks, y

In [6]:
class FeedbackProblem(MyProblem):
    def pred_dim(self):
        return [0,3]

In [12]:
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', 2550)
df_train = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
df_test = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")
df_train["text"] = df_train["essay_id"].apply(lambda x: open(f'../input/feedback-prize-effectiveness/train/{x}.txt').read())
df_test["text"] = df_test["essay_id"].apply(lambda x: open(f'../input/feedback-prize-effectiveness/test/{x}.txt').read())
effectiveness_map = {"Ineffective":0, "Adequate":1,"Effective":2}
df_train["target"] = df_train["discourse_effectiveness"].map(effectiveness_map)
labels = np.zeros((len(df_train), 3))
labels[np.arange(len(df_train)), df_train['target'].values] = 1
labels

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [69]:
problem_type = FeedbackProblem()
model_type = MyBertModel(debug=False)

In [70]:
kfold_model_inproc(model_fold_storage, problem_type, model_type, df_train, labels, df_test)

kfold 15532


5it [00:00, 1000.12it/s]


loaded model bert from models/bert_0_model
['.gitattributes', 'LICENSE', 'README.md', 'config.json', 'flax_model.msgpack', 'pytorch_model.bin', 'rust_model.ot', 'tf_model.h5', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


0it [00:00, ?it/s]

230/230 [==============================] - 27s 113ms/step
['.gitattributes', 'LICENSE', 'README.md', 'config.json', 'flax_model.msgpack', 'pytorch_model.bin', 'rust_model.ot', 'tf_model.h5', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


...

0it [00:00, ?it/s]

459/459 [==============================] - 27s 56ms/step - loss: 0.7167 - accuracy: 0.6880
['.gitattributes', 'LICENSE', 'README.md', 'config.json', 'flax_model.msgpack', 'pytorch_model.bin', 'rust_model.ot', 'tf_model.h5', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


0it [00:00, ?it/s]

1/1 [==============================] - 0s 199ms/step


(array([[0.17421667, 0.80860215, 0.01718116],
        [0.28198764, 0.70575362, 0.01225873],
        [0.23532078, 0.73372877, 0.03095045],
        ...,
        [0.10895097, 0.80916321, 0.08188581],
        [0.3918609 , 0.58889937, 0.01923972],
        [0.27587071, 0.6970889 , 0.02704039]]),
 array([[[0.27095884, 0.6877656 , 0.04127556],
         [0.1978251 , 0.75418776, 0.04798716],
         [0.16098413, 0.69339514, 0.14562067],
         [0.54613298, 0.43652976, 0.01733723],
         [0.34396839, 0.63820982, 0.01782181],
         [0.39779875, 0.56405592, 0.03814538],
         [0.44721809, 0.51481909, 0.03796284],
         [0.3368617 , 0.62902337, 0.03411494],
         [0.30910653, 0.63142329, 0.05947017],
         [0.05629733, 0.39520109, 0.54850161]],
 
        [[0.21908961, 0.76013273, 0.02077775],
         [0.16819073, 0.80227941, 0.02952984],
         [0.22762035, 0.67723095, 0.09514876],
         [0.60704732, 0.38445744, 0.00849524],
         [0.25874647, 0.70254529, 0.03870823],
 

In [ ]:
all_metrics